In [1]:
import numpy as np 
import pandas as pd
import collections # counter
import seaborn as sns
from imblearn.over_sampling import SMOTE, ADASYN # oversampling
from copy import *
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler # encoding categorical data, feature scaling
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV, train_test_split # splitting training and testing data
from sklearn.datasets import make_classification
from sklearn.utils import column_or_1d
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict

In [11]:
# folder = 'csv_he'
# folder = 'csv_pp_500'
folder = 'csv_b_500'
# selection_method = 1
selection_method = 2

In [12]:
df_users = pd.read_csv(folder + '/df_users_columns_' + str(selection_method) + '.csv')

In [17]:
df_users[df_users['repeat_buyer'] == True]

,n_sessions,repeat_buyer,n_top_product_u,n_unique_product_u,n_top_category_u,n_unique_category_u,user_mean_eff_duration,mean_price,n_pageviews_buy,n_pageviews_no_buy,...,continent,device.type,browser.name,os.name,referrer1,time_2,time_3,time_1,day_1,day_2
7,2,True,3,10,8,14,106634.500000,0,25.0,0.0,...,EU,desktop,Chrome,Windows,barefootky,0,2,0,2,0
12,10,True,6,26,16,10,94038.750000,0,48.0,12.0,...,EU,mobile,Chrome,Android,google,3,1,0,3,1
23,73,True,5,31,12,20,98653.000000,0,38.0,34.0,...,EU,mobile,Chrome,Android,google,2,2,0,2,2
24,6,True,5,16,9,13,140580.000000,0,12.0,57.0,...,EU,desktop,Chrome,Windows,google,1,1,1,1,2
25,7,True,4,40,35,5,131908.333333,0,38.0,21.0,...,EU,mobile,Chrome,Android,google,2,3,1,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
870,12,True,10,6,7,2,93798.333333,0,24.0,4.0,...,EU,mobile,-,Android,-,1,0,2,3,0
872,9,True,4,9,5,14,68479.800000,0,15.0,12.0,...,EU,desktop,Firefox,Windows,google,2,3,0,3,2
878,3,True,4,8,8,27,243918.500000,0,34.0,0.0,...,EU,desktop,Firefox,Windows,barefootky,0,2,0,2,0
879,3,True,7,14,12,2,106254.000000,0,25.0,29.0,...,EU,mobile,-,Android,google,2,0,1,3,0


In [4]:
labeled_column = 'repeat_buyer'
useful_columns = [
    'n_pageviews_buy', 'n_pageviews_no_buy',
    'n_input_buy', 'n_input_no_buy', 
    'n_click_buy', 'n_click_no_buy', 
    'n_mouse_click_buy', 'n_mouse_click_no_buy', 
    'n_mouse_move_buy', 'n_mouse_move_no_buy',
    'n_scroll_move_buy', 'n_scroll_move_no_buy', 
    'n_scrandom_buy', 'n_scrandom_no_buy', 
    'n_events_buy', 'n_events_no_buy', 
    'n_product_buy', 'n_product_no_buy',
    'n_non_product_buy', 'n_non_product_no_buy', 
    'n_category_buy', 'n_category_no_buy', 
    'n_filter_buy', 'n_filter_no_buy', 
    'n_search_buy', 'n_search_no_buy', 
    'n_cart_buy', 'n_cart_no_buy', 
    'n_add_to_cart_buy', 'n_add_to_cart_no_buy', 
#     'n_remove_from_cart_buy', 'n_remove_from_cart_no_buy',
#     'n_next_page_buy', 'n_next_page_no_buy',
    'effective_duration_buy', 'effective_duration_no_buy', 
    'pv_product_mean_eff_duration_buy', 'pv_product_mean_eff_duration_no_buy',
    'user_mean_eff_duration',
    'load_time_buy', 'load_time_no_buy', 
    'n_sessions',
    'n_top_product_u', 'n_unique_product_u',
    'n_top_category_u', 'n_unique_category_u',
    'country', 'city_type', 'continent', 
    'device.type', 'browser.name', 'os.name',
    'referrer1',
    'time_1',
    'time_2', 
    'time_3', 
    'day_1', 
    'day_2',
    'mean_price', 
#     'mean_rate',  
]
numeric_columns = [
    'n_pageviews_buy', 'n_pageviews_no_buy',
    'n_input_buy', 'n_input_no_buy', 
    'n_click_buy', 'n_click_no_buy', 
    'n_mouse_click_buy', 'n_mouse_click_no_buy', 
    'n_mouse_move_buy', 'n_mouse_move_no_buy',
    'n_scroll_move_buy', 'n_scroll_move_no_buy', 
    'n_scrandom_buy', 'n_scrandom_no_buy', 
    'n_events_buy', 'n_events_no_buy', 
    'n_product_buy', 'n_product_no_buy',
    'n_non_product_buy', 'n_non_product_no_buy', 
    'n_category_buy', 'n_category_no_buy', 
    'n_filter_buy', 'n_filter_no_buy', 
    'n_search_buy', 'n_search_no_buy', 
    'n_cart_buy', 'n_cart_no_buy', 
    'n_add_to_cart_buy', 'n_add_to_cart_no_buy', 
#     'n_remove_from_cart_buy', 'n_remove_from_cart_no_buy',
#     'n_next_page_buy', 'n_next_page_no_buy',
    'effective_duration_buy', 'effective_duration_no_buy', 
    'pv_product_mean_eff_duration_buy', 'pv_product_mean_eff_duration_no_buy',
    'user_mean_eff_duration',
    'n_sessions',
    'load_time_buy', 'load_time_no_buy', 
    'n_top_product_u', 'n_unique_product_u',
    'n_top_category_u', 'n_unique_category_u',
    'mean_price', 
#     'mean_rate',
    'time_1',
    'time_2', 
    'time_3', 
    'day_1', 
    'day_2',
]

categorical_columns = [c for c in useful_columns if c not in numeric_columns]

In [5]:
# Encoding categorical features
encoding_method = 1

# One-hot encoding - 0/1 values
if (encoding_method == 1):
    encoded_data = pd.DataFrame()

    for column in categorical_columns:
        dummies = pd.get_dummies(df_users[column], prefix = column) # one-hot encoding - function get_dummies()
        encoded_data = pd.concat([encoded_data, dummies], axis = 'columns')

# Label encoding - na 0, ..., n - 1 values
elif (encoding_method == 2):
    encoded_data = pd.DataFrame()

    for category in categorical_columns:
        a = pd.DataFrame({category: df_users[category].astype('category').cat.codes})
        encoded_data = pd.concat([encoded_data, a], axis = 'columns')

# join with numeric columns
encoded_data = pd.concat([encoded_data, df_users[numeric_columns]], axis='columns')

In [6]:
encoded_data.columns

Index(['country_AR', 'country_AT', 'country_CZ', 'country_DE', 'country_DK',
       'country_EE', 'country_FI', 'country_FR', 'country_HU', 'country_IE',
       'country_IT', 'country_NL', 'country_SI', 'country_SK', 'country_US',
       'city_type_0', 'city_type_1', 'city_type_2', 'city_type_3',
       'continent_-', 'continent_EU', 'continent_SA', 'device.type_desktop',
       'device.type_mobile', 'device.type_tablet', 'browser.name_-',
       'browser.name_Chrome', 'browser.name_Edge', 'browser.name_Facebook',
       'browser.name_Firefox', 'browser.name_MIUI Browser',
       'browser.name_Opera', 'browser.name_Safari', 'os.name_-',
       'os.name_Android', 'os.name_Linux', 'os.name_Windows', 'os.name_iOS',
       'referrer1_-', 'referrer1_azet', 'referrer1_barefootky',
       'referrer1_bing', 'referrer1_centrum', 'referrer1_facebook',
       'referrer1_google', 'referrer1_instagram', 'n_pageviews_buy',
       'n_pageviews_no_buy', 'n_input_buy', 'n_input_no_buy', 'n_click_buy',


In [7]:
pd.options.display.max_columns = None
# cv = False 
cv = True
k = 5

# train/test set size (%)
if cv == False:
    test_size = 0.20
    k = 1

In [8]:
# train/test set split
X_train, X_test, Y_train, Y_test = list(), list(), list(), list()
if cv == False:
    x_train, x_test, y_train, y_test = train_test_split(encoded_data, df_users[labeled_column], test_size=test_size, random_state=0)
    X_train.append(x_train)
    X_test.append(x_test)
    Y_train.append(y_train)
    Y_test.append(y_test)
    k = 1
    
if cv == True:
    kf = KFold(n_splits=5)
    k = kf.get_n_splits(encoded_data)
    X = encoded_data
    Y = df_users[labeled_column]
    for train_index, test_index in kf.split(X):
        X_train.append(X.iloc[train_index])
        X_test.append(X.iloc[test_index])
        Y_train.append(Y.iloc[train_index])
        Y_test.append(Y.iloc[test_index])

In [9]:
for i in range(k):
    print('Train data - number of records: ' + str(len(X_train[i])))
    print('Test data - number of records: ' + str(len(X_test[i])))
    print('Number of conversions: ')
    print(collections.Counter(Y_train[i]))
    print('--------------------------------------')

Train data - number of records: 723
Test data - number of records: 181
Number of conversions: 
Counter({False: 584, True: 139})
--------------------------------------
Train data - number of records: 723
Test data - number of records: 181
Number of conversions: 
Counter({False: 587, True: 136})
--------------------------------------
Train data - number of records: 723
Test data - number of records: 181
Number of conversions: 
Counter({False: 569, True: 154})
--------------------------------------
Train data - number of records: 723
Test data - number of records: 181
Number of conversions: 
Counter({False: 571, True: 152})
--------------------------------------
Train data - number of records: 724
Test data - number of records: 180
Number of conversions: 
Counter({False: 561, True: 163})
--------------------------------------


In [10]:
numeric_train_data = pd.DataFrame(X_train[0][numeric_columns])

In [11]:
# normalization
def normalization_sampling(data, label, normalization_method, sampling_method):

    # split data by type
    num_cols = [column for column in numeric_columns if column in X_train[i].columns]
    numeric_data = pd.DataFrame(data[num_cols])

    normalized_data = numeric_data
            
    if (normalization_method == 1):
        # values <0, 1>
        for column in numeric_columns:
            if column in X_train[i].columns:
                normalized_data[column] = pd.DataFrame((numeric_data[column] - min(numeric_train_data[column])) / (max(numeric_train_data[column]) - min(numeric_train_data[column])))
            
    elif (normalization_method == 2):
        # Z-score
        for column in numeric_columns:
            if column in X_train[i].columns:
                normalized_data[column] = (numeric_data[column] - numeric_train_data[column].mean()) / numeric_train_data[column].std()
        
    data = data.drop(columns = num_cols)
    data = pd.concat([data, normalized_data], axis='columns')
    
    # Sampling - SMOTE
    if (sampling_method == 1):
        data, label = SMOTE().fit_resample(data, label)
        
    return data, label

In [12]:
for i in range(k):
    X_train[i], Y_train[i] = normalization_sampling(X_train[i], Y_train[i], 2, 1)
    X_test[i], Y_test[i] = normalization_sampling(X_test[i], Y_test[i], 2, 0)

In [13]:
for i in range(k):
    print('Number of conversions: ')
    print(collections.Counter(Y_train[i]))

Number of conversions: 
Counter({False: 584, True: 584})
Number of conversions: 
Counter({False: 587, True: 587})
Number of conversions: 
Counter({False: 569, True: 569})
Number of conversions: 
Counter({False: 571, True: 571})
Number of conversions: 
Counter({False: 561, True: 561})


In [ ]:
# Export CSV
for i in range(0,k):
    X_train[i].to_csv(folder + '/train_data_' + str(selection_method) + '_cv_' + str(cv) + '_' + str(i) + '.csv', index=False, encoding='utf-8-sig')
    X_test[i].to_csv(folder + '/test_data_' + str(selection_method) + '_cv_' + str(cv) + '_' + str(i) + '.csv', index=False, encoding='utf-8-sig')

    Y_train[i].to_csv(folder + '/train_label_' + str(selection_method) + '_cv_' + str(cv) + '_' + str(i) + '.csv', index=False, encoding='utf-8-sig')
    Y_test[i].to_csv(folder + '/test_label_' + str(selection_method) + '_cv_'+ str(cv) + '_' + str(i) + '.csv', index=False, encoding='utf-8-sig')